In [7]:
from final_utils import *
import shap

In [8]:
root_path = generate_root_path()

In [9]:
def fc_model_softmax(input_num=16928):
    input_ = Input(shape=(input_num,))
    x = Dense(2048, kernel_initializer='he_normal', activation='relu')(input_)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Dense(1024, kernel_initializer='he_normal', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Dense(256, kernel_initializer='he_normal', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    pred = Dense(2, activation='softmax')(x)

    model = Model(input_,pred)
    return model

### Creating model

In [10]:
model = fc_model_softmax(input_num=19488)
model.load_weights('models/Softmax/MG_CNN/model_fc_softmax_16k_2k.hdf5')

### Creating dataset


In [ ]:
paths_multigap = glob(f'{root_path}Data/splitted/train/features/mg/original/*')
paths_cnn = glob(f'{root_path}Data/splitted/train/features/cnn_efficientnet_b7/border_600x600/*')

In [ ]:
for i in range(len(paths_multigap)):
    basename_mg = (os.path.basename(paths_multigap[i]).split('.'))[0] 
    basename_cnn = (os.path.basename(paths_cnn[i]).split('.'))[0] 
    if basename_mg != basename_cnn:
        print('false')
    if i % 100 == 0:
#         print(i)
        break
print('true')

In [ ]:
feator_vectors = []
for i in range(len(paths_multigap)):
    mg_vector = np.load(paths_multigap[i])
    cnn_vector = np.load(paths_cnn[i])
    feator_vector = np.concatenate((mg_vector,cnn_vector), axis  = 1)
    feator_vectors.append(feator_vector)
    if i == 100:
        break
    if i % 1000 == 0:
        print(i)
print(len(feator_vectors))
feator_vectors = np.asarray(feator_vectors)
feator_vectors = np.squeeze(feator_vectors)
print(feator_vectors.shape)


In [11]:
main_path = root_path + 'Data/splitted/train/'
feats_MG = 'original' 
feats_CNN = 'border_600x600'
cnn = 'cnn_efficientnet_b7'

# New way to load feature vectors (json)

In [12]:
def feature_vectors_load_json(main_path=main_path, feats_mg=feats_MG, feats_cnn=feats_CNN, cnn=cnn):
    vectors_list = []
    labels_list = []
 
    for i in range(7):
        alm_train_bad = open(f'{main_path}data_bad{i+1}.json')
        bad_data = json.load(alm_train_bad)
        for data in bad_data:
            feat_path_1 = main_path + f'features/mg/{feats_MG}/' + data['feature']
            feat_path_2 = main_path + f'features/{cnn}/{feats_CNN}/' + data['feature']
            print(f'good data label = ' + data['label'])
            connected = np.concatenate((np.squeeze(np.load(feat_path_1)), np.squeeze(np.load(feat_path_2))))
            vectors_list.append(connected)
            labels_list.append(data['label'])
    alm_train_good = open(f'{main_path}/data_good1.json')
    good_data = json.load(alm_train_good)
    for data in good_data:
        feat_path_1 = main_path + f'features/mg/{feats_MG}/' + data['feature']
        feat_path_2 = main_path + f'features/{cnn}/{feats_CNN}/' + data['feature']
        print(f'good data label = ' + data['label'])
        connected = np.concatenate((np.squeeze(np.load(feat_path_1)), np.squeeze(np.load(feat_path_2))))
        vectors_list.append(connected)
        labels_list.append(data['label'])
    
    vectors_array = np.squeeze(np.array(vectors_list))
    labels_array = np.squeeze(np.array(labels_list))
    
    return vectors_array, labels_array
    


In [13]:
features = feature_vectors_load_json()

good data label = bad
good data label = bad
good data label = bad
good data label = bad
good data label = bad
good data label = bad
good data label = bad
good data label = good


### Explanation with SHAP 

In [ ]:
explainer = shap.GradientExplainer(model, feator_vectors)

In [ ]:
shap_values = []
for i in range(feator_vectors.shape[0]):
    shap_values.append(explainer.shap_values(feator_vectors[i])[np.argmax(model.predict(feator_vectors[i]))][0])####
    if i % 5000 == 0:
        print(i)
    
len(shap_values)

In [ ]:

def principal_values_1(data, num_samples):
    data_index = np.argsort(data, axis = 1)[:,::-1]
#     print(f'data_index = {data_index}')
    data_slice = data_index[:,:500] #Attention to this !!!!
#     print(f'data_slice = {data_slice}')
    hist, bins = np.histogram(data_slice, bins = data.shape[1] + 1, range = (0, data.shape[1] + 1))
#     print(type(hist))
    sorted_vector = np.sort(hist)[::-1]

#     print(f'hist, bins = {hist, bins}')
    indxs = np.argsort(hist)[::-1]
#     print(f'indxs = {indxs}')
    return indxs[:num_samples] , sorted_vector[:num_samples]

In [ ]:
def take_from_vector(feature_vector, indxs):
    return feature_vector[indxs]